#### Imports/setup

In [1]:
from parameters.config_imports import *

In [2]:
def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

#### Inputs : feature collection (without Geo IDs)

From asset

In [3]:
asset_name = "example_asset"

feature_collection_asset_id = f"projects/{gee_cloud_project}/assets/{asset_name}"

In [4]:
roi_asset = ee.FeatureCollection(feature_collection_asset_id)

From Geojson

In [5]:
file_path_geojson = "input_examples/example_geojson.geojson"

roi_geojson = geojson_to_ee(file_path_geojson)

Choose input

In [6]:
# comment out one to select input
# roi = roi_asset
roi = roi_geojson
roi


In [7]:
# temp code: uncomment below line if want to test using point inputs based on polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over >500 polygons currently this will go to Google Drive and require manual uploading here
- Stats sent to a separate notebook 'process_whisp_stats.pynb'

In [8]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
Datasets present in lookup:  True
geo id not present ['Plot_area_ha', 'Geometry_type', 'Country', 'Centroid_lon', 'Centroid_lat', 'Unit']
...stats processing started
...ee to df processing time: 4.341159343719482 seconds


### Display it


In [9]:
df

,Plot_area_ha,Geometry_type,Country,Centroid_lon,Centroid_lat,Unit,EUFO_2020,GLAD_Primary,TMF_undist,JAXA_FNF_2020,...,TMF_loss_before_2020,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,TMF_deg_after_2020,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,1.938,Polygon,GHA,-1.611942,6.159539,percent,19,0,0,100,...,23,80,0,0,0,0,0,0,0,0
1,16.602,Polygon,GHA,-2.157144,5.981149,percent,100,87,95,100,...,0,0,0,0,0,0,0,0,6,1
2,31.210,Polygon,IDN,103.956096,-3.054668,percent,0,0,0,6,...,100,99,100,21,0,0,0,0,0,2
3,20.884,Polygon,IDN,103.977512,-3.083808,percent,28,0,31,96,...,15,43,0,0,0,0,0,0,0,3
4,8.279,Polygon,CIV,-4.101646,5.711935,percent,5,0,0,95,...,32,39,0,0,0,0,19,0,0,4
5,3.613,Polygon,CIV,-7.507022,6.071468,percent,41,0,0,100,...,27,54,0,0,0,0,0,0,0,5


#### Export it to CSV file
NB skip if exporting to Google Drive

In [10]:
#export output csv
print  (out_csv)
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)

/home/sepal-user/module_results/whisp/whisp_output_table.csv
output csv:  /home/sepal-user/module_results/whisp/whisp_output_table.csv


#### Add Geo ids
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- In the meantime to stop timeouts extend session length click dollar sign and edit under 'Sessions'.
- If processing doesn't complete, rerun and it should pickup from where you got to.

In [11]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)

Chosen path for temp lookup csv:/home/sepal-user/module_results/whisp/temp_geo_id_lookup.csv 

Check passed: feature collection and table same size: 6 rows
 Joins may not be working - please log this in 'Issues' in github
Number without geo ids:0
All geo ids present in lookup csv
Done 



##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [12]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

new csv:  /home/sepal-user/module_results/whisp/whisp_output_table_w_geo_id.csv


##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [13]:
# remove_column_from_csv(
#     csv_file="copy_whisp_output_table.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [14]:
# Replace with the name of your CSV file
csv_file =out_directory/'whisp_output_table_w_geo_id.csv'

# import as a dataframe
df_w_stats = pd.read_csv(csv_file)


df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )



##### Display table with risk columns

In [15]:
df_w_risk

,Plot_area_ha,Geo_id,Geometry_type,Country,Centroid_lon,Centroid_lat,Unit,EUFO_2020,GLAD_Primary,TMF_undist,...,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,1.938,NaN,Polygon,GHA,-1.611942,6.159539,percent,19,0,0,...,0,0,0,0,0,yes,no,yes,no,low
1,16.602,NaN,Polygon,GHA,-2.157144,5.981149,percent,100,87,95,...,0,0,0,6,1,yes,no,yes,yes,low
2,31.210,NaN,Polygon,IDN,103.956096,-3.054668,percent,0,0,0,...,0,0,0,0,2,yes,yes,yes,no,low
3,20.884,NaN,Polygon,IDN,103.977512,-3.083808,percent,28,0,31,...,0,0,0,0,3,yes,no,yes,no,low
4,8.279,NaN,Polygon,CIV,-4.101646,5.711935,percent,5,0,0,...,0,19,0,0,4,yes,yes,yes,yes,low
5,3.613,NaN,Polygon,CIV,-7.507022,6.071468,percent,41,0,0,...,0,0,0,0,5,yes,yes,yes,no,low


##### Export table with risk columns to csv 

In [16]:
output_file=out_directory/'whisp_output_table_w_geo_id_w_risk.csv'

df_w_risk.to_csv(output_file)
